In [2]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import math
import pandas as pd
from numpy import genfromtxt

# Functions

In [10]:
def cleanData(t,dup,col1):
    
    #get rid of duplicates
    t.mask(dup == True)
    
    cols = np.array([])
    #get rid of duplicates
    imask = 0
    for i in range(len(col1)):
        first_match = col1[i]
        for j in range(i+1,len(col1)):
            if (first_match == col1[j]):
                cols = np.append(cols,j)
            else: break
    cols = cols.astype(int)
    t.drop(t.index[cols])
        
    return t

In [23]:
def AbsoluteMagnitudes(plx,Bmag,Vmag,Gmag,Jmag,Hmag,Kmag,umag,gmag,rmag,imag,zmag):
    #Calculate distance, and absolute magnitudes
    
    Dist = np.array([])
    Bmags = np.array([])
    Vmags = np.array([])
    Gmags = np.array([])
    Jmags = np.array([])
    Hmags = np.array([])
    Kmags = np.array([])
    umags = np.array([])
    gmags = np.array([])
    rmags = np.array([])
    imags = np.array([])
    zmags = np.array([])

    Dist = 1000./plx
    Bmags = Bmag - 5 * np.log10(Dist) + 5
    Vmags = Vmag - 5 * np.log10(Dist) + 5
    Gmags = Gmag - 5 * np.log10(Dist) + 5
    Jmags = Jmag - 5 * np.log10(Dist) + 5
    Hmags = Hmag - 5 * np.log10(Dist) + 5
    Kmags = Kmag - 5 * np.log10(Dist) + 5
    umgas = umag - 5 * np.log10(Dist) + 5
    gmags = gmag - 5 * np.log10(Dist) + 5
    rmags = rmag - 5 * np.log10(Dist) + 5
    imags = imag - 5 * np.log10(Dist) + 5
    zmags = zmag - 5 * np.log10(Dist) + 5
    
    return Dist, Bmags, Vmags, Gmags,Jmags,Hmags,Kmags, umgas, gmags, rmags, imags, zmags

In [12]:
def ColorClassification(Gmags,Kmags,colors):    
    #split the stars into different categories based on G-K
    
    G_K = Gmags-Kmags
    color = np.ones(len(G_K))*1000
    for i in range(len(colors)):
        for j in range(len(G_K)):
            if (i == 0):
                if (G_K[j] <= colors[i]):
                    color[j] = colors[i]
            elif(G_K[j] <= colors[i] and G_K[j] > colors[i-1]):
                color[j] = colors[i]
    return color

In [14]:
def CreateTemplates(color,colors,wvl,wvl2,Bmags, Vmags,Gmags,Jmags,Hmags,Kmags,umags,gmags,rmags,imags,zmags):
    #for each color class, a template is created of the absolute magnitudes of the stars and their color indices
    
    G_K = Gmags-Kmags
    avg_colors = np.array([])
    err_colors = np.array([])
    stars = np.array([])
    for c in colors:
        #create array of values for color indices and absolute magnitudes
        G_B_array = np.array([])
        G_V_array = np.array([])
        G_J_array = np.array([])
        G_H_array = np.array([])
        G_K_array = np.array([])
        G_u_array = np.array([])
        G_g_array = np.array([])
        G_r_array = np.array([])
        G_i_array = np.array([])
        G_z_array = np.array([])
        B = np.array([])
        V = np.array([])
        G = np.array([])
        J = np.array([])
        H = np.array([])
        K = np.array([])
        u = np.array([])
        g = np.array([])
        r = np.array([])
        i = np.array([])
        z = np.array([])
        star_count = 0
        #fill arrays with values if the values are part of the color category
        for i in range(len(G_K)):
            if (color[i] == c):
                G_B_array = np.append(G_B_array,Gmags[i]-Bmags[i])
                G_V_array = np.append(G_V_array,Gmags[i]-Vmags[i])
                G_J_array = np.append(G_J_array,Gmags[i]-Jmags[i])
                G_H_array = np.append(G_H_array,Gmags[i]-Hmags[i])
                G_K_array = np.append(G_K_array,Gmags[i]-Kmags[i])
                G_u_array = np.append(G_u_array,Gmags[i]-umags[i])
                G_g_array = np.append(G_g_array,Gmags[i]-gmags[i])
                G_r_array = np.append(G_r_array,Gmags[i]-rmags[i])
                G_i_array = np.append(G_i_array,Gmags[i]-umags[i])
                G_z_array = np.append(G_z_array,Gmags[i]-zmags[i])
                B = np.append(B,Bmags[i])
                V = np.append(V,Vmags[i])
                G = np.append(G,Gmags[i])
                J = np.append(J,Jmags[i])
                H = np.append(H,Hmags[i])
                K = np.append(K,Kmags[i])
                u = np.append(u,umags[i])
                g = np.append(g,umags[i])
                r = np.append(r,rmags[i])
                i = np.append(i,imags[i])
                z = np.append(z,zmags[i])
                star_count = star_count +1
        #average G-K value
        avg_colors = np.append(avg_colors,np.mean(G_K_array))
        err_colors = np.append(err_colors,np.std(G_K_array))
        #number of stars per color
        stars = np.append(stars,star_count)
        #relative SED
        mag_array = [np.mean(G_B_array), np.mean(G_V_array), \
                     np.mean(G_J_array), np.mean(G_H_array), np.mean(G_K_array), \
                     np.mean(G_u_array), np.mean(G_g_array), np.mean(G_r_array), \
                     np.mean(G_i_array), np.mean(G_z_array)]
        std_array = [np.std(G_B_array), np.std(G_V_array), \
                     np.std(G_J_array), np.std(G_H_array), np.std(G_K_array), \
                     np.std(G_u_array), np.std(G_g_array), np.std(G_r_array), \
                     np.std(G_i_array), np.std(G_z_array)]
        #absolute SED
        mag_array2 = [np.mean(B), np.mean(V), np.mean(G), \
                      np.mean(J), np.mean(H), np.mean(K), \
                      np.mean(u), np.mean(g), np.mean(r), \
                      np.mean(i), np.mean(z)]
        std_array2 = [np.std(B), np.std(V), np.std(G), \
                      np.std(J), np.std(H), np.std(K), \
                      np.std(u), np.std(g), np.std(r), \
                      np.std(i), np.std(z)]
        #add standard deviation error
        
        val = np.sqrt(len(G_B_array))
        std_array3 = std_array/val
        val2 = np.sqrt(len(B))
        std_array4 = std_array2/val2
        
        #stack SEDs
        if (c == colors[0]):
            color_mags = np.array(mag_array)
            color_std = np.array(std_array)
            color_std_err = np.array(std_array3)
            abs_color_mags = np.array(mag_array2)
            abs_color_std = np.array(std_array2)
            abs_color_std_err = np.array(std_array4)
        else:
            color_mags = np.vstack((color_mags,mag_array))
            color_std = np.vstack((color_std,std_array))
            color_std_err = np.vstack((color_std_err,std_array3))
            abs_color_mags = np.vstack((abs_color_mags, mag_array2))
            abs_color_std = np.vstack((abs_color_mags, std_array2))
            abs_color_std_err = np.vstack((abs_color_std_err,std_array4))
    return stars,color_mags,color_std,color_std_err,abs_color_mags,abs_color_std,abs_color_std_err

# Read in Data and Create Templates

In [ ]:
t = pd.read_csv('../Cat_GAIA200pc_2MASS_SDSS_v1_XmatchSimbad.csv')

print("The reader loads the data into the following variable names:")
print(list(t))
print("\nThe number of stars is " + str(len(t)))

dup = t['duplicated_source'] 
col1 = t["col1"]

#Clean Data
t = cleanData(t,dup,col1)

Source = t["source_id"]
#Using Gmag in place of Vmag and Bmag to calculate Vmags and Bmags
Bmag = t["phot_g_mean_mag"]
Vmag = t["phot_g_mean_mag"]
Gmag = t["phot_g_mean_mag"]
Jmag = t["j_m"]
Hmag = t["h_m"]
Kmag = t["ks_m"]
umag = t["u_mag"]
gmag = t["g_mag"]
rmag = t["r_mag"]
imag = t["i_mag"]
zmag = t["z_mag"]
plx = t['plx']
dup = t['duplicated_source'] 
col1 = t["col1"]

Dist,Vmags,Bmags,Gmags,Jmags,Hmags,Kmags,umags,gmags,rmags,imags,zmags = \
AbsoluteMagnitudes(plx,Vmag,Bmag,Gmag,Jmag,Hmag,Kmag,umag,gmag,rmag,imag,zmag)

In [27]:
wvl = [420, 477, 1235, 1662, 2159]
wvl2 = [420, 477, 673,1235, 1662, 2159]
colors = [0.5,1,1.5,2,2.5,3] 

color = ColorClassification(Gmags,Kmags,colors)
stars,color_mags,color_std,color_std_err,abs_color_mags,abs_color_std,abs_color_std_err = \
CreateTemplates(color,colors,wvl,wvl2,Vmags,Bmags,Gmags,Jmags,Hmags,Kmags,umags,gmags,rmags,imags,zmags)